In [1]:
import os
import json
import re

In [2]:
root_dir = '/Users/egeozin/desktop/civic_data/gehl/vision/yolo/'
json_dir = '/Users/egeozin/desktop/civic_data/gehl/vision/yolo/tested.json'

myre = re.compile(r"\w+(?=.*:)[a-zA-Z]")

full_dir = os.path.join(root_dir, 'test.txt')

texts = tuple(open(full_dir, 'r'))

texts = [t.replace('\n', '') for t in texts]

labels = []

for i in texts:
    dicto = {}
    with open(i) as myfile:
        data = myfile.readlines()
        data = [d.replace('\n', '') for d in data]
        dicto[i.replace('labels/', '').replace('.txt','')] = data
        labels.append(dicto)
        
        
with open(json_dir) as f_in:
    jso = json.load(f_in) 

In [10]:
a  = [ 1, 2, 3, 4, 2, 3]

b = set(a)

for i in b:
    print(i)

1
2
3
4


In [12]:
# false positives, false negatives, true positives, true negatives
# Defined by Classification with Heterogeneous Features (CHF)  [Godbole et al, 2004]
# Excluding Intersection over Union accuracy models for bounding boxes
# Only multi-label classification.
# Precision

lookup = {'0':'bench', '1':'sit', '2':'stand', '3':'backrest'}

accuracy = 0
precision = 0
recall = 0

n = 200

def unite_truth_and_pred(truth, pred):
    truth_copy = list(truth)
    pred_copy = list(pred)
    n_items = len(truth_copy)
    n_preds = len(predicted)
    n_union = 0
    if pred_copy:
        #if len(truth) >= len(pred):
        #if n_items >= n_preds:
        #    count = n_items
        #    pred_set = set(pred_copy)
        #    for p in pred_set:
        #        if not p in truth_copy:
        #            count += 1
        #    return count
        #else:
        #    count = n_preds
        #    truth_set = set(truth_copy)
        #    for t in truth_set:
        #        if not t in pred_copy:
        #            count += 1
        #    return count
        if n_items >= n_preds:
            count = 0
            for p in pred_copy:
                if p in truth_copy:
                    count +=1
                    truth_copy.remove(p)
            count += len(truth_copy)
            return count
        else:
            count = 0
            for t in truth_copy:
                if t in pred_copy:
                    count += 1
                    pred_copy.remove(t)
            count += len(pred_copy)
            return count
    else:
        return 0
                         
for i in labels:
    truth = [lookup[t] for t in [k.split(" ", 1)[0] for k in i.items()[0][1]] if not t == '-1'] # [0][1][0].split(" ", 1)[0]
    n_items = len(truth)
    predicted = myre.findall(jso[i.items()[0][0]])
    n_preds = len(predicted)
    n_union = unite_truth_and_pred(truth, predicted)
    if predicted:
        this_score = 0
        count = 0
        for p in predicted:
            if count < n_items:
                count += 1
                if p in truth:
                    this_score += 1
                    truth.remove(p)
        
        accuracy += this_score/float(n_union)
        precision += this_score/float(n_items)
        recall += this_score/float(n_preds)

In [13]:
# The proportion of predicted correct labels to the total number of labels(predicted or actual), 
# averaged over all instances

# 0.663598665224

accuracy = accuracy/n
print accuracy

# The proportion of predicted correct labels to the total number of actual labels, 
# averaged over all instances

# 0.769722222222

precision = precision/n
print precision

# The proportion of predicted correct labels to the total number of predicted labels, 
# averaged over all instances

# 0.788320887446

recall = recall/n
print recall

0.663598665224
0.769722222222
0.788320887446
